In [2]:
import torch
import torch.nn as nn
from torch.functional import F
from utils import *
from static_model import *
from temporal_model import *
from global_variables import *

In [5]:
def load_encoder(path='./models/', name="checkpoint1.state_dict", **kwargs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = StaticModel(eye_params, face_params, face_grid_params).to(device).eval()
    state_dict = torch.load(path + name, map_location=device)
    model.load_state_dict(state_dict)

    return nn.Sequential(*list(model.children())[:-1])

In [9]:
class CNNtoRNN(nn.Module):
	"""docstring for CNNtoRNN"""
	def __init__(self, ins, hs, nl, **kwargs):
		super(CNNtoRNN, self).__init__()
		# load encoder
		# train sequence model
		# concatenate + fc
		# out
		self.encoder = load_encoder(**kwargs)
		self.encoder.freeze()
		self.decoder = SequenceModel(ins, hs, nl, **kwargs)

		self.fc = linear(128*2, 128, True, 0.5)
		self.out = nn.Linear(128, 2)

	def forward(self, xfs, xls, xrs, xgs):
		xe = self.encoder(xfs, xls, xrs, xgs)#[seq_len, 128]
		xd = self.decoder(xe)#[seq_len, 128]
		x = torch.cat((xd, xe), 1)

		x = self.fc(x)
		return self.out(x)

In [10]:
model = CNNtoRNN(256, 256, 4)

FileNotFoundError: [Errno 2] No such file or directory: './models/checkpoint1.state_dict'